In [ ]:
# if your are using DCM images
!pip install pydicom

CONVERT DCM TO JPG

In [ ]:
import os
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import numpy as np
from google.colab import drive
# Define input and output folders
input_folder_path = '/path/to/dcm'
output_folder_path = '/path/to/jpg'
# Create the output folder if it doesn't exist
os.makedirs(output_folder_path, exist_ok=True)
# Function to convert DICOM to RGB
def dicom_to_rgb(dcm_path):
  # Read DICOM file
  dcm = pydicom.dcmread(dcm_path)
  # Apply VOI LUT (if present)
  image = apply_voi_lut(dcm.pixel_array, dcm)
  # Convert to 8-bit unsigned integer
  image = ((image / image.max()) * 255).astype(np.uint8)
  # Stack the image to create an RGB image
  rgb_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
  return rgb_image
# Iterate through DICOM files in the input folder
for filename in os.listdir(input_folder_path):
  if filename.endswith('.DCM'):
    dcm_path = os.path.join(input_folder_path, filename)
    # Convert DICOM to RGB
    rgb_image = dicom_to_rgb(dcm_path)
    # Save as JPG in the output folder
    jpg_path = os.path.join(output_folder_path, os.path.splitext(filename)[0] + '.jpg')
    cv2.imwrite(jpg_path, cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR))

In [ ]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)
labelmap_path = '/path/to/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)
output_directory = '/path/to/faster_rcnn_inference_graph'
tf.keras.backend.clear_session()
model = tf.saved_model.load(f'{output_directory}/saved_model')
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy()
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

  return output_dict

LOAD YOUR MODELS

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.models import load_model
molar_premolar_model = load_model('/path/to/molar_premolar_model.h5')
overlap_premolar_model = load_model('/path/to/overlap_premolar_model.h5')
overlap_molar_model = load_model('/path/to/overlap_molar_model.h5')
decayed_premolar_model = load_model('/path/to/decayed_premolar_model.h5')
decayed_molar_model = load_model('/path/to/decayed_molar_model.h5')

PATH YOUR BITEWING JPG IMAGES DIRECTORY TO PREDICT

In [ ]:
path = "/path/to/jpg_images"
for filename in os.listdir(path):
  for image_path in glob.glob(os.path.join(path, filename)):
    image_np = load_image_into_numpy_array(image_path)
    output_dict = run_inference_for_single_image(model, image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True,
        line_thickness=1)
    display(Image.fromarray(image_np))

In [ ]:
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from google.colab.patches import cv2_imshow
import glob
# Function to crop teeth based on bounding boxes
direction = []
def crop_teeth(image, boxes, classes, scores, category_index):
  cropped_teeth = []
  for i in range(len(boxes)):
    if scores[i] > 0.5:
      box = tuple(boxes[i].tolist())
      ymin, xmin, ymax, xmax = box
      if ymin < 0.2:
        direction.append('up')
      else:
        direction.append('down')
      ymin, xmin, ymax, xmax = int(ymin * image.shape[0]), int(xmin * image.shape[1]), int(ymax * image.shape[0]), int(xmax * image.shape[1]) # Crop the tooth region from the image
      tooth_cropped = image[ymin:ymax, xmin:xmax]
      newsize = (224, 224)
      tooth_cropped = cv2.resize(tooth_cropped, newsize)
      cropped_teeth.append(tooth_cropped)
  return cropped_teeth
c = -1
path = "/path/to/bitewing_jpg_images"
for filename in os.listdir(path):
  image_np = load_image_into_numpy_array(os.path.join(path, filename))
  output_dict = run_inference_for_single_image(model, image_np)
  # Crop teeth based on bounding boxes
  cropped_teeth = crop_teeth(image_np, output_dict['detection_boxes'], output_dict['detection_classes'], output_dict['detection_scores'], category_index)
  # Now 'cropped_teeth' is a list containing cropped images of detected teeth
  for i, tooth_cropped in enumerate(cropped_teeth):
    x = tooth_cropped.copy()
    plt.imshow(x)
    plt.show()
    c += 1
    tooth_cropped = preprocess_input(tooth_cropped)
    # tooth_cropped = np.array(tooth_cropped)
    tooth_cropped = np.expand_dims(tooth_cropped, axis=0)
    test_predictions = molar_premolar_model.predict(tooth_cropped)
    predicted_labels = np.round(test_predictions).astype(int).flatten()
    print('premolar' if predicted_labels == 0 else 'molar')
    if predicted_labels == 1:
      # tooth_cropped = cv2.resize(tooth_cropped[0], (134, 224))
      # tooth_cropped = tooth_cropped.reshape(1, 224, 134, 3)
      test_predictions = overlap_molar_model.predict(tooth_cropped)
      predicted_labels = np.argmax(test_predictions, axis=1)
      print('overlapped' if predicted_labels == 0 else 'non-overlapped' if predicted_labels==1 else 'useless')
      if predicted_labels == 1:
        x = x.astype(np.uint8)
        gray_image = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(4, 4))
        enhanced_image = clahe.apply(gray_image)
        alpha = 1.5
        result_image = cv2.convertScaleAbs(enhanced_image, alpha=alpha, beta=0)
        ribbon_width = int(0.3 * result_image.shape[1])
        ribbon_position = result_image.shape[1] // 2 - ribbon_width // 2
        result_image[:, ribbon_position:ribbon_position + ribbon_width] = 0
        height, width = result_image.shape
        upper_half = result_image[:height//3, :]
        lower_half = result_image[height//3 * 2:, :]
        if 'up' == direction[c]:
          upper_half[:, :] = 0
        else:
          lower_half[:, :] = 0
        image = cv2.cvtColor(result_image, cv2.COLOR_GRAY2RGB)
        tooth_cropped = tf.keras.applications.resnet.preprocess_input(image)
        # tooth_cropped = np.array(tooth_cropped)
        tooth_cropped = np.expand_dims(tooth_cropped, axis=0)
        tooth_cropped = cv2.resize(tooth_cropped[0], (224, 224))
        tooth_cropped = tooth_cropped.reshape(1, 224, 224, 3)
        test_predictions = decayed_molar_model.predict(tooth_cropped)
        predicted_labels = np.round(test_predictions).astype(int).flatten()
        print('non-decayed' if predicted_labels == 0 else 'decayed')
    else:
      tooth_cropped = cv2.resize(tooth_cropped[0], (134, 224))
      tooth_cropped = tooth_cropped.reshape(1, 224, 134, 3)
      test_predictions = overlap_premolar_model.predict(tooth_cropped)
      predicted_labels = np.argmax(test_predictions, axis=1)
      print('overlapped' if predicted_labels == 0 else 'non-overlapped' if predicted_labels==1 else 'useless')
      if predicted_labels == 1:
        x = x.astype(np.uint8)
        gray_image = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(4, 4))
        enhanced_image = clahe.apply(gray_image)
        alpha = 1.5
        result_image = cv2.convertScaleAbs(enhanced_image, alpha=alpha, beta=0)
        ribbon_width = int(0.3 * result_image.shape[1])
        ribbon_position = result_image.shape[1] // 2 - ribbon_width // 2
        result_image[:, ribbon_position:ribbon_position + ribbon_width] = 0
        height, width = result_image.shape
        upper_half = result_image[:height//3, :]
        lower_half = result_image[height//3 * 2:, :]
        if 'up' == direction[c]:
          upper_half[:, :] = 0
        else:
          lower_half[:, :] = 0
        print(direction[c-1])
        image = cv2.cvtColor(result_image, cv2.COLOR_GRAY2RGB)
        tooth_cropped = tf.keras.applications.resnet.preprocess_input(image)
        # tooth_cropped = np.array(tooth_cropped)
        tooth_cropped = np.expand_dims(tooth_cropped, axis=0)
        tooth_cropped = cv2.resize(tooth_cropped[0], (134, 224))
        tooth_cropped = tooth_cropped.reshape(1, 224, 134, 3)
        test_predictions = decayed_premolar_model.predict(tooth_cropped)
        predicted_labels = np.round(test_predictions).astype(int).flatten()
        print('non-decayed' if predicted_labels == 0 else 'decayed')